# ITOS - Intraday Trading Operating System

## Complete Workflow Demo

This notebook demonstrates the complete ITOS workflow:
1. **Data Generation** - Create sample market data
2. **Strategy Execution** - Run strategies across symbols
3. **Analytics** - Evaluate strategy performance
4. **Decision Making** - Approve/Modify/Kill strategies

In [ ]:
# Install dependencies (run once)
!pip install pandas numpy pyarrow matplotlib seaborn tqdm jupyter

In [ ]:
# Import ITOS modules
import sys
from pathlib import Path

# Add project to path
project_root = Path().absolute()
sys.path.insert(0, str(project_root))

from itos.core.data_engine import DataEngine
from itos.core.strategy_engine import StrategyEngine
from itos.core.analytics_engine import AnalyticsEngine
from itos.core.sample_data_generator import SampleDataGenerator
from itos.strategies.str001_orb import STR001_ORB
from itos.strategies.str002_orb_filtered import STR002_ORB_FILTERED
from itos.strategies.str003_vwap_pullback import STR003_VWAP_PULLBACK

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ ITOS modules imported successfully")

## Step 1: Generate Sample Data

In [ ]:
# Create sample data generator
generator = SampleDataGenerator()

# Generate 3 months of data for 5 symbols
print("🔧 Generating sample data...")
generator.generate_sample_data(
    start_date='2023-01-01',
    end_date='2023-03-31',
    symbols=['RELIANCE', 'TCS', 'HDFCBANK', 'INFY', 'HINDUNILVR']
)

print("✅ Sample data generated")

# Verify data
data_engine = DataEngine()
symbols = data_engine.get_processed_symbols()
print(f"📊 Processed symbols: {symbols}")

## Step 2: Preview Sample Data

In [ ]:
# Load and preview data for one symbol
symbol = 'RELIANCE'
market_data = data_engine.load_symbol_data(symbol)

if market_data:
    print(f"📈 Data for {symbol}:")
    print(f"   Shape: {market_data.data.shape}")
    print(f"   Date range: {market_data.data.index.min()} to {market_data.data.index.max()}")
    print(f"   Columns: {list(market_data.data.columns)}")
    
    # Display first few rows
    display(market_data.data.head())
    
    # Plot price chart
    plt.figure(figsize=(12, 6))
    plt.plot(market_data.data.index, market_data.data['close'])
    plt.title(f'{symbol} - Price Chart')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print(f"❌ No data found for {symbol}")

## Step 3: Run Strategies

In [ ]:
# Initialize strategy engine
strategy_engine = StrategyEngine()

# Define strategies to run
strategies = [
    STR001_ORB(),
    STR002_ORB_FILTERED(),
    STR003_VWAP_PULLBACK()
]

print("🚀 Running strategies...")

# Run each strategy
for strategy in strategies:
    strategy_name = strategy.get_config().name
    print(f"\n📊 Running {strategy_name}...")
    
    success = strategy_engine.run_strategy(strategy)
    
    if success:
        # Load and display results
        trades_df = strategy_engine.get_strategy_results(strategy_name)
        if trades_df is not None and not trades_df.empty:
            print(f"   ✅ {strategy_name} completed with {len(trades_df)} trades")
            print(f"   Sample trades:")
            display(trades_df.head())
        else:
            print(f"   ⚠️ {strategy_name} completed but no trades generated")
    else:
        print(f"   ❌ {strategy_name} failed")

## Step 4: Analytics & Evaluation

In [ ]:
# Initialize analytics engine
analytics_engine = AnalyticsEngine()

print("📈 Running analytics...")

# Analyze each strategy
results = []

for strategy in strategies:
    strategy_name = strategy.get_config().name
    print(f"\n🔍 Analyzing {strategy_name}...")
    
    # Load trades
    trades_df = strategy_engine.get_strategy_results(strategy_name)
    
    if trades_df is not None and not trades_df.empty:
        # Run analytics
        result = analytics_engine.analyze_strategy(strategy_name, trades_df)
        results.append(result)
        
        # Print summary
        print(f"\n📋 {strategy_name} Summary:")
        print(f"   Total Trades: {result.total_trades}")
        print(f"   Winning Trades: {result.winning_trades}")
        print(f"   Losing Trades: {result.losing_trades}")
        print(f"   Total PnL: ₹{result.total_pnl:,.2f}")
        print(f"   Win Rate: {result.winning_trades/result.total_trades*100:.1f}%")
        print(f"   Profit Factor: {result.profit_factor:.2f}")
        print(f"   Max Drawdown: {result.max_drawdown*100:.2f}%")
        print(f"   Sharpe Ratio: {result.sharpe_ratio:.2f}")
        print(f"   Avg Duration: {result.avg_trade_duration:.1f} minutes")
        
        # Get recommendation
        recommendation = analytics_engine.evaluate_strategy(result)
        print(f"   Recommendation: {recommendation}")
        
        if recommendation == "APPROVE":
            print("   ✅ Strategy approved for live trading")
        elif recommendation == "MODIFY":
            print("   ⚠️ Strategy needs modification")
        else:
            print("   ❌ Strategy killed - not viable")
    else:
        print(f"   ⚠️ No trades found for {strategy_name}")

## Step 5: Strategy Comparison

In [ ]:
# Compare strategies
if results:
    strategy_names = [r.strategy_name for r in results]
    comparison_df = analytics_engine.compare_strategies(strategy_names)
    
    print("\n📊 Strategy Performance Comparison:")
    display(comparison_df)
    
    # Create comparison chart
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Strategy Comparison Dashboard', fontsize=16)
    
    # Total PnL comparison
    axes[0, 0].bar(comparison_df['Strategy'], comparison_df['Total PnL'])
    axes[0, 0].set_title('Total PnL')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # Win Rate comparison
    axes[0, 1].bar(comparison_df['Strategy'], comparison_df['Win Rate'] * 100)
    axes[0, 1].set_title('Win Rate (%)')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Profit Factor comparison
    axes[1, 0].bar(comparison_df['Strategy'], comparison_df['Profit Factor'])
    axes[1, 0].set_title('Profit Factor')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # Sharpe Ratio comparison
    axes[1, 1].bar(comparison_df['Strategy'], comparison_df['Sharpe Ratio'])
    axes[1, 1].set_title('Sharpe Ratio')
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ No strategy results to compare")

## Step 6: Detailed Trade Analysis

In [ ]:
# Analyze trades for best performing strategy
if results:
    # Find best strategy by total PnL
    best_result = max(results, key=lambda x: x.total_pnl)
    best_strategy_name = best_result.strategy_name
    
    print(f"\n🏆 Best Strategy: {best_strategy_name}")
    print(f"   Total PnL: ₹{best_result.total_pnl:,.2f}")
    
    # Load trades for detailed analysis
    trades_df = strategy_engine.get_strategy_results(best_strategy_name)
    
    if trades_df is not None and not trades_df.empty:
        # Convert entry_time to datetime
        trades_df['entry_time'] = pd.to_datetime(trades_df['entry_time'])
        trades_df['exit_time'] = pd.to_datetime(trades_df['exit_time'])
        
        # Trade analysis
        print(f"\n📊 Detailed Trade Analysis for {best_strategy_name}:")
        
        # PnL distribution
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.hist(trades_df['pnl'], bins=30, alpha=0.7)
        plt.title('PnL Distribution')
        plt.xlabel('PnL')
        plt.ylabel('Frequency')
        
        # Cumulative PnL
        plt.subplot(1, 2, 2)
        trades_df_sorted = trades_df.sort_values('entry_time')
        trades_df_sorted['cumulative_pnl'] = trades_df_sorted['pnl'].cumsum()
        plt.plot(trades_df_sorted['entry_time'], trades_df_sorted['cumulative_pnl'])
        plt.title('Cumulative PnL')
        plt.xlabel('Date')
        plt.ylabel('Cumulative PnL')
        plt.xticks(rotation=45)
        
        plt.tight_layout()
        plt.show()
        
        # Exit reason analysis
        exit_reason_stats = trades_df.groupby('exit_reason').agg({
            'pnl': ['count', 'sum', 'mean'],
            'symbol': 'nunique'
        }).round(2)
        
        print(f"\n📋 Exit Reason Analysis:")
        display(exit_reason_stats)
        
        # Symbol performance
        symbol_stats = trades_df.groupby('symbol').agg({
            'pnl': ['count', 'sum', 'mean'],
        }).round(2)
        
        print(f"\n📈 Symbol Performance:")
        display(symbol_stats.head(10))
else:
    print("❌ No strategy results to analyze")

## Summary & Next Steps

In [ ]:
print("\n" + "="*60)
print("🎉 ITOS Demo Completed!")
print("="*60)

if results:
    print(f"\n📊 Summary:")
    print(f"   Strategies Tested: {len(results)}")
    print(f"   Total Trades Generated: {sum(r.total_trades for r in results)}")
    print(f"   Best Strategy: {max(results, key=lambda x: x.total_pnl).strategy_name}")
    print(f"   Total PnL: ₹{sum(r.total_pnl for r in results):,.2f}")
    
    # Count recommendations
    approved = sum(1 for r in results if analytics_engine.evaluate_strategy(r) == 'APPROVE')
    modify = sum(1 for r in results if analytics_engine.evaluate_strategy(r) == 'MODIFY')
    kill = sum(1 for r in results if analytics_engine.evaluate_strategy(r) == 'KILL')
    
    print(f"\n🎯 Recommendations:")
    print(f"   ✅ Approved: {approved}")
    print(f"   ⚠️ Modify: {modify}")
    print(f"   ❌ Kill: {kill}")

print(f"\n📁 Generated Files:")
print(f"   📊 Data: itos/minute_parquet/")
print(f"   📈 Results: itos/STR_results/")
print(f"   📋 Analytics: itos/analytics/")
print(f"   📄 Comparison: itos/final/strategy_comparison.csv")

print(f"\n🔄 Next Steps:")
print(f"   1. Review detailed analytics in itos/analytics/")
print(f"   2. Examine strategy dashboards (PNG files)")
print(f"   3. Modify/kill underperforming strategies")
print(f"   4. Develop new strategies using the framework")
print(f"   5. Test with real market data")

print("\n" + "="*60)